In [29]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import os
from rasterio.windows import Window
from rasterio.plot import reshape_as_raster, reshape_as_image
from glob import glob

In [30]:
def normalize_rgb(tile):
    tile = tile[[2, 1, 0],:,:].astype(np.float64)
    max_val = 4000
    min_val = 0
    # Enforce maximum and minimum values
    tile[tile[:, :, :] > max_val] = max_val
    tile[tile[:, :, :] < min_val] = min_val
    for b in range(tile.shape[0]):
        tile[b, :, :] = tile[b, :, :] * 1 / (max_val - min_val)
    tile_reshaped = reshape_as_image(tile)
    return tile_reshaped

In [31]:
def make_combined(tile_number, ls8_filepaths, dates):
    j = 0
    for ls8_fp in ls8_filepaths:
        ls8_band_paths = glob(ls8_fp + '*SRB*')
        ls8_band_paths.sort()
        ls8_band_paths += glob(ls8_fp + '*PIXELQA*')
        img_fp = '/deep_data/recurrent_data/' + 'tile' + tile_number + '/' + 'combined/' + 'combined' +dates[j]+'.tif'
        j+=1
        # Read metadata of first file and assume all other bands are the same
        with rasterio.open(ls8_band_paths[0]) as src0:
            meta = src0.meta
        # Update metadata to reflect the number of layers
        meta.update(count = len(ls8_band_paths)+1) #add 1 because ndvi
        # Read each layer and write it to stack
        with rasterio.open(img_fp, 'w', **meta) as dst:
            for id, layer in enumerate(ls8_band_paths, start=1):
                print(layer)
                if id < 7:
                    with rasterio.open(layer) as src1:
                        dst.write_band(id, src1.read(1))
                        if id == 5:
                            red = src1.read(1)
                        elif id == 6:
                            nir = src1.read(1)
                else:
                    ndvi = (nir.astype(float) - red.astype(float)) / (nir.astype(float) + red.astype(float) + 0.0000001)
                    ndvi *= 10000
                    dst.write_band(id, ndvi.astype(np.int16))
                    with rasterio.open(layer) as src1:
                        dst.write_band(id+1, src1.read(1).astype(np.int16)) # this one otherwise is uint16 for some reason    

In [32]:
def view_landsat(landsat):
    image_dataset = landsat
    full_img = landsat.read()
    colors_reshaped = normalize_rgb(full_img)
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(colors_reshaped)
    ax.set_title("RGB in matplotlib imshow")

In [33]:
tiles = {}
tiles['028012'] = ['20110324', '20110612', '20110831', '20111103']
tiles['029011'] = ['20110308', '20110425', '20110831', '20111103']
tiles['028011'] = ['20110308', '20110628', '20110831', '20111103']

In [ ]:
for tile_number, dates in tiles.items():
    ls8_filepaths = []
    for date in dates:
        ls8_filepaths.append('/deep_data/recurrent_data/tile'+tile_number+'/date'+date+'/')
    make_combined(tile_number, ls8_filepaths, dates)

In [34]:
landsat_datasets = {}
for tile_number, dates in tiles.items():
    tile_datasets = []
    l8_image_paths = []
    for date in dates:
        l8_image_paths.append('/deep_data/recurrent_data/tile{}/combined/combined{}.tif'.format(tile_number, date))
    for fp in l8_image_paths:
        tile_datasets.append(rasterio.open(fp))
    landsat_datasets[tile_number] = tile_datasets

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(landsat_datasets['029011'][0].read()[6])
ax.set_title("RGB in matplotlib imshow")
view_landsat(landsat_datasets['029011'][0])


In [ ]:
view_landsat(landsat_datasets['028012'][0])
view_landsat(landsat_datasets['028012'][1])
view_landsat(landsat_datasets['028012'][2])
view_landsat(landsat_datasets['028012'][3])

In [35]:
tiles = {}
landsat_datasets = {}
tiles['028012'] = ['20110324', '20110612', '20110831', '20111103']
tiles['029011'] = ['20110308', '20110425', '20110831', '20111103']
tiles['028011'] = ['20110308', '20110628', '20110831', '20111103']
for tile_number, dates in tiles.items():
    tile_datasets = []
    l8_image_paths = []
    for date in dates:
        l8_image_paths.append('/deep_data/recurrent_data/tile{}/combined/combined{}.tif'.format(tile_number, date))
    for fp in l8_image_paths:
        tile_datasets.append(rasterio.open(fp))
    landsat_datasets[tile_number] = tile_datasets

In [8]:
lc_labels = rasterio.open('/deep_data/recurrent_data/NLCD_DATA/landcover/NLCD_2011_Land_Cover_L48_20190424.img')
canopy_labels = rasterio.open('/deep_data/recurrent_data/NLCD_DATA/canopy/CONUSCartographic_2_8_16/Cartographic/nlcd2011_usfs_conus_canopy_cartographic.img')
#terio.open('/deep_data/recurrent_data/NLCD_DATA/canopy/CONUSAnalytical_2_8_16/Analytical/nlcd2011_usfs_conus_canopy_analytical.img')


In [57]:
from rnn_tiles import rnn_tile_gen
import importlib
import rnn_pixels
importlib.reload(rnn_pixels)

<module 'rnn_pixels' from '/host/Desktop/cnn_dev/florence_mapping/rcnn/rnn_pixels.py'>

In [43]:
pixels = rnn_pixels.gen_pixel_locations(landsat_datasets, 10000, 32)

In [44]:
print(len(pixels))

100020


In [45]:
new_px = rnn_pixels.delete_bad_tiles(landsat_datasets, pixels, 32)

In [46]:
print(len(new_px))

43325


In [87]:
import rnn_tiles
importlib.reload(rnn_tiles)

<module 'rnn_tiles' from '/host/Desktop/cnn_dev/florence_mapping/rcnn/rnn_tiles.py'>

In [88]:
gen = rnn_tiles.rnn_tile_gen(landsat_datasets, lc_labels, canopy_labels, 32, 16)

In [110]:
generator = gen.tile_generator(new_px, 25)

In [90]:
print("{}".format(lc_labels.height))
print("{}".format(lc_labels.width))

104424
161190


In [91]:
print(r)

[[  0   0   0 ... 255 255 255]
 [  0   0   0 ... 255 255 255]
 [  0   0   0 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]


In [113]:
l8, labels = next(generator)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[ 6  1  6 ...  0  0  0]
 [ 5  5 11 ...  0  0  0]
 [12  4  2 ...  0  0  0]
 ...
 [ 9 12 14 ...  7  5  5]
 [20 16 17 ... 16 13 16]
 [18 24 26 ... 15 19 36]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 

In [ ]:
l8 = l8.reshape((4, 32, 32, 7))

In [ ]:
lc_label['landcover'].shape

In [ ]:
labels = lc_label['canopy'].reshape(32, 32, 1)

In [ ]:
labels.shape

In [ ]:
zero = Proj(landsat_datasets['028012'][0].crs)
one = Proj(landsat_datasets['028012'][3].crs)

In [ ]:
new_px[0][0][0]

In [ ]:
from pyproj import Proj, transform